## P2 Exploration

Basic exploration will be done in this jupyter notebook.

In [ ]:
from helpers import *
 
# Just fill this with wherever is your main data folder is (copy the path just before 'MovieSummaries'):
file_source = "/Users/shrinidhivelan/Downloads/"

# Same goes for the reviews dataset : but simply copy the entire path of where your reviews data is... 
reviews_path = "/Users/shrinidhivelan/Downloads/TMDB_movie_dataset_v11.csv"

### creating dataset by extracting 
MovieMetadata_df, CharacterMetadata_df, names_df, plot_summaries_df, tvTropes_df, merged_df = create_datasets(file_source, reviews_path)